In [ ]:
import nltk
import re
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
import numpy as np
import warnings

In [ ]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))
wv = api.load('word2vec-google-news-300')

In [ ]:
df = pd.read_csv(r"/kaggle/input/review-data/review_data3.csv")

df.head()

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace = True)

In [ ]:
df.shape

In [ ]:
sns.histplot(x = df['tokens'].apply(len))
plt.xlabel('length_of_tokens')

In [ ]:
def tokens_to_padded_embeddings_list(tokens_list, max_sequence_length = 500):
    embeddings_list = []
    for tokens in tokens_list:
        embeddings = []
        for token in tokens:
            if token in wv.key_to_index:
                embedding = wv[token]
                embeddings.append(embedding)
        embeddings_list.append(np.array(embeddings))

    # Pad sequences to a common length
    padded_embeddings_list = pad_sequences(embeddings_list, maxlen=max_sequence_length, padding='post', dtype='float32')
    return padded_embeddings_list

In [ ]:
train = df['tokens']
train_label = df['sentiment']



In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(train, train_label, test_size = 0.15, random_state = 42, shuffle = True)

In [ ]:
x_train = tokens_to_padded_embeddings_list(X_train.to_list())

In [ ]:
x_valid = tokens_to_padded_embeddings_list(X_valid.to_list())

In [ ]:
x_valid.shape

In [ ]:
x_train.shape

In [ ]:
#def create_bilstm_model():
    model = Sequential([])

    # Add Bidirectional LSTM layer
    
    
    # Define the Bidirectional LSTM model
    model = Sequential()

    model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(500, 300)))
    model.add(layers.Dropout(0.2))

    model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
    model.add(layers.Dropout(0.2))

    model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['f1'])

    return model

In [ ]:
def create_bilstm_model():
    model = Sequential()

    # Add Bidirectional LSTM layer
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(500, 300)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.3))

    # Add output layer
    model.add(Dense(1, activation='sigmoid'))  # Binary classification

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['f1'])

    return model

In [ ]:

model = create_bilstm_model()
model.summary()

In [ ]:
history = model.fit(x_train, Y_train,
                    validation_data=(x_valid, Y_valid),
                    batch_size=32, epochs=10)